# Melting

**Source:** Wagner, T. J., Dell, R. W., & Eisenman, I. (2017). An Analytical Model of Iceberg Drift. *Journal of Physical Oceanography*.

## Melt Terms

### Wind-driven wave erosion:

\begin{equation}
M_e = 0.5 S_s = 0.75 \mid \vec{v_a} - \vec{v_w} \mid^{0.5} + 0.05 \mid \vec{v_a} - \vec{v_w} \mid
\end{equation}


### Turbulent basal melt:

\begin{equation}
M_b = 0.58 \mid \vec{v_w} - \vec{v_i} \mid^{0.8} (T_w - T_i)L^{-0.2}
\end{equation}


### Buoyant convection (thermal side-wall erosion):

\begin{equation}
M_v = 0.0076 T_w + 0.0013 T_w^2
\end{equation}


#### Notes:

- *Other processes such as surface melt, have been found to be small compared to these terms (Savage 2001).*
- *Water temperature, $T_w$, is approximated by the SST.*
- *Iceberg temperature, $T_i$, is assumed constant at $T_i=-4^\circ C$. (El Tahan et. al. 1987)*
- *All temperatures are in degrees celcius and L is in meters*
- *Each of the melt terms, $M-E$, $M_b$, and $M_v$, are expressed in meters of change in iceberg dimensions per day*
- *We assume that these processes are linearly additive, such that iceberg volume evolves as: *$\frac{dV}{dt} = \frac{d(LWH)}{dt}$


## Melt Rates

$$\frac{dL}{dt} = \frac{dW}{dt} = -M_v - M_e \quad \frac{dH}{dt} = -M_b$$

$$L^{i+1} = L^{i} + \frac{dL}{dt} \quad W^{i+1} = W^{i} + \frac{dW}{dt} \quad  H^{i+1} = H^{i} + \frac{dH}{dt}$$

## Rollovers

### Aspect ratio:

$$\epsilon \equiv \frac{W}{H}$$

### Critical value:

$$\epsilon_c = \sqrt{6\frac{\rho_i}{\rho_w}(1 - \frac{\rho_i}{\rho_w})}$$

*below which a rectangular iceberg is unconditionally unstable.* 

#### Assumptions:
*Assuming $\rho_i = 850 kg/m^3$ (based on tabular density for icebergs in the Southern Ocean) and a water density of $\rho_w = 1025 kg/m^3$ we get:*

$$\frac{\rho_i}{\rho_w} = 0.83$$

$$\epsilon_c = 0.92$$

In [1]:
# Modules

import numpy as np
import numpy.matlib
import time
import scipy.io as sio
from scipy.spatial import cKDTree as ckd

In [2]:
def tic():
    #Homemade version of matlab tic and toc functions
    global startTime_for_tictoc
    startTime_for_tictoc = time.time()

def toc():
    import time
    if 'startTime_for_tictoc' in globals():
        print("Elapsed time is {} seconds.".format(str(time.time() - startTime_for_tictoc)))
    else:
        print("Toc: start time not set")
        
# Source: http://stackoverflow.com/questions/5849800/tic-toc-functions-analog-in-python/5849861
# Credit: GuestUser

In [3]:
# Analytic Parameters

R = 6378 * 1e3  ## earth radius in m
rhow = 1027  ## density of water (kg/m^3)
rhoa = 1.2  ## density of air   (kg/m^3)
rhoi = 850  ## density of ice   (kg/m^3)
drho = rhow - rhoi
Cw = 0.9  ## bulk coefficient water  (Bigg et al 1997)
Ca = 1.3  ## bulk coefficient air    (Bigg et al 1997)
om = 7.2921 * 1e-5  ## rotation rate of earth (rad/s)
g = np.sqrt(rhoa * drho / rhow / rhoi * (Ca / Cw))  ## gamma = np.sqrt(ca/cw)


# Melt parameters

Ti0 = -4
Cs1 = 1.5 
Cs2 = 0.5 
Cs3 = 0.1
CMv1 = 7.62e-3 
CMv2 = 1.29e-3
CMe1 = 0.5
CMb1 = 0.58 
CMb2 = 0.8 
CMb3 = 0.2
    

In [4]:
def a(U):
    # \alpha in the paper
    a = np.sqrt(2) / np.power(U, 3) * (1 - np.sqrt(1 + np.power(U, 4)))
    return a

def b(U):
    # \beta in the paper
    b = np.real(1 / np.power(U, 3) * np.sqrt(1 + np.power(U, 4)) - 3 * np.power(U, 4) - 4)
    return b

def ff(lati):
    # Latitude in degrees
    ff = 2 * om * sin(abs(lati) * pi/180)
    return ff

def S(l, w):
    # Harmonic mean length
    S = pi * l * w / (l + w)
    return S

def Ut(u, lati, S):
    # \Lambda in the papers
    Ut = np.sqrt(2) * Cw * g / np.multiply(ff(lati), u/s)
    return Ut


In [5]:
# Set directories

modelfull = 'ECCO_20th'
modelshort = 'E2'
root = '/home/evankielley/WagnerModel'
condloc = root + '/conditions/' + modelfull + '/'
outloc = root + '/output/' + modelfull + '/'
modelloc = root + '/Model/'

In [14]:
# Load input fields

tic()
mask = sio.loadmat(condloc + 'mask.mat')
vel = sio.loadmat(condloc + modelshort + '_vels_1992.mat')
vel = vel['vel']
sst = sio.loadmat(condloc + modelshort +'_sst_1992.mat')
sst = sst['sst']
bergdims = sio.loadmat(modelloc + 'bergdims.mat')
bergdims = bergdims['bergdims']

print('Model data loaded \n')

toc()

Model data loaded 

Elapsed time is 2.8299074172973633 seconds.


In [112]:
# Load Seeding fields

tic()

Laurent_Seed = sio.loadmat(modelloc + 'Laurent_Seed.mat')
Seed_X = Laurent_Seed['Seed_X']; Seed_Y = Laurent_Seed['Seed_Y']

# Cycle through each location 100x (i.e. this can run 3600 icebergs)
seed_X = np.matlib.repmat(Seed_X[:], 100, 1); seed_Y = np.matlib.repmat(Seed_Y[:], 100, 1)

print('Fields seeded \n')

toc()

Fields seeded 

Elapsed time is 0.0021593570709228516 seconds.


In [113]:
# Specify the space domain

tic()

LAT = vel['latw'] * 1.0 
LAT = LAT[0,0]
LON = vel['lonw'] * 1.0
LON = LON[0,0]
minLAT = min(LAT[:]) 
maxLAT = max(LAT[:])
minLON = min(LON[:]) 
maxLON = max(LON[:])

print('Space domain specified \n')

toc()

Space domain specified 

Elapsed time is 0.010023117065429688 seconds.


In [115]:
# Set run parameters 

trajnum = 25            # total number of iceberg trajectories to compute
final_t = 122           # number of input field time steps
startrange = final_t / 2  # input field start range
tres = 3                # time resoln such that "model Dt"="input DT"/tres
DT = 3                  # Input fields time step
Dt = DT / tres            # model timestep in days
dt = Dt * 24 * 3600         # model timestep in seconds
R = 6378 * 1e3            # earth radius in m
dtR = dt / R * 180 / np.pi       # need this ratio for distances in "drifting.m"
t = np.arange(1, final_t)      # how long is the run
nt= (t.size) * tres             # number of model timesteps
tt = np.linspace(1, t.size, nt) # model time

In [116]:
# Set the circulation fields

tic()

## water vels input
#uwF = vel.uw[:,:,t]; vwF = vel.vw[:,:,t] 
uwF = vel['uw']; vwF = vel['vw']


## air vels input
#uaF = vel.ua[:,:,t]; vaF = vel.va[:,:,t]  
uaF = vel['ua']; vaF = vel['va']  


## sst vels input
#sst = sst[:,:,t]     

print('Circulation fields set \n')

toc()

Circulation fields set 

Elapsed time is 0.0005519390106201172 seconds.


In [117]:
def construct_seeding():
    
    glacier = 'H'   #pick the glacier you want to construct the seeding for

    if glacier == 'H':
        # Helheim Ice Sheet Seeding Locations ---------------------------------
        Glac_Y = np.arange(259,262,1)  #[261,190] 66.3500 N, 38.2000 W coord, Helheim
        Glac_X = np.arange(188,192,1) 
        glac_name = 'Helheim'
        
    elif glacier == 'J':
        # Jakobshaven Ice Sheet Seeding Locations -----------------------------
        Glac_Y = np.arange(276,280,1)  #[277,120] 69 10 N 49 50 W coord, Jakobshavn
        Glac_X = np.arange(119,123,1) 
        glac_name = 'Jakobsh'
        
    elif glacier == 'K':
        # Kangerd Ice Sheet Seeding Locations ---------------------------------
        Glac_Y = np.arange(267,271,1)  #[277,120] 68 38 N 33 0 W coord, Kangerd
        Glac_X = np.arange(213,1,218) 
        glac_name = 'Kangerd'
        
    elif glacier == 'L': 
        # Laurentide Ice Sheet Seeding Locations ------------------------------
        Glac_Y = np.arange(245,255,2)  #Laurentide LAT seed locations (in ECCO 2 grid)
        Glac_X = np.arange(86,101,2)    #Laurentide LON seed locations (in ECCO 2 grid)
        glac_name = 'Laurent' 
    

    seed_LAT = [LAT[Glac_Y[0]]*[1, 1], LAT[Glac_Y[-1]]*[1, 1]] 
    seed_LON = np.matlib.repmat([LON[Glac_X[0]], LON[Glac_X[-1]]],[2,1]) 
    seed_LAT = np.matlib.repmat([LAT[Glac_Y[0]], LAT[Glac_Y[-1]]],[2,1]) 
    seed_LON = [LON[Glac_X[0]]*[1, 1], LON[Glac_X[-1]]*[1, 1]] 
    Seed_Y, Seed_X = meshgrid(Glac_Y, Glac_X) 
        

In [118]:
def melting(i):
    
    I = i
    
    # Melt terms
    
    Me = CMe1 * (Cs1 * Ua^Cs2 + Cs3 * Ua)  ## Wind driven erosion
    Mv = CMv1 * SST + CMv2 * SST^2  ## Thermal side wall erosion 
    Mb = CMb1 * np.sqrt((ui - uw)^2 + (vi - vw)^2)^CMb2 * (SST - Ti0) / l(I)^CMb3  ## Turbulent basal melt 

    # Melt rates
    
    dldt = - Mv - Me 
    dhdt = - Mb
    l[I+1] = l[I] + dldt * Dt  
    w[I+1] = w[I] + dldt * Dt  
    h[I+1] = h[I] + dhdt * Dt 
    
    
    # Check if iceberg size is negative
    
    if l[I+1] < 0 | w[I+1] < 0 | h[I+1] < 0:
        
        l[I+1] = 0 
        w[I+1] = 0 
        h[I+1] = 0
        melted = 1   ## Boolean
        mm = mm + 1   ## Counter for icebergs that have melted
    
    
    # Rollover
    
    if w[I+1] < 0.85 * h[I+1]:
        hn = w[I+1]  ## new height
        w[I+1] = h[I+1] 
        h[I+1] = hn
    
    
    # Check if length is greater than width
    
    if w[I+1] > l[I+1]:
        wn = l[I+1] 
        l[I+1] = w[I+1] 
        w[I+1] = wn
    
    
    # New volume and change in volume (dv)
    
    v[I+1] = l[I+1] * w[I+1] * h[I+1]
    dv[I+1] = v(I) - v[I+1]
    
    
    # Check if iceberg survived
    
    if I == lt-1 & v[I+1] > 0:
        ss = ss + 1   ## counter for icebergs that have survived
    
    
    # Store melt rates
    
    Mev = Me 
    Mvv = Mv 
    Mbv = Mb
    

In [119]:
def drifting(i):
    
    I = i
    
    # Find nearest neighbour
    ## CAUTION: this only works on a rectangular grid!
    
    YI = ckd(LAT, yil[I])
    XI = ckd(LON, xil[I])
    
    
    # Interpolate fields linearly between timesteps
    
    timestep = tt[tts + I]
    t1  = floor(timestep) 
    t2 = t1 + 1
    dt1 = timestep - t1 
    dt2 = t2 - timestep
    ua = uaF[XI,YI,t1] * dt1 + uaF[XI,YI,t2] * dt2 
    va = vaF[XI,YI,t1] * dt1 + vaF[XI,YI,t2] * dt2 
    uw = uwF[XI,YI,t1] * dt1 + uwF[XI,YI,t2] * dt2 
    vw = vwF[XI,YI,t1] * dt1 + vwF[XI,YI,t2] * dt2 
    SST= sst[XI,YI,t1] * dt1 + sst[XI,YI,t2] * dt2 
    
    
    # Compute wind speed and "U tilde" at location for a given iceberg size
    
    Ua = np.sqrt(ua^2+va^2)
    UT = Ut(Ua, yil[I], S(l[I],w[I]))  ## Ut and S are functions
    
    
    # Iceberg translation
    ## Note the conversion from meters to degrees lon/lat
    
    dlon = ui * dtR 
    dlat = vi * dtR 

    uiv[I] = ui 
    viv[I] = vi
    
    uav[I] = ua 
    vav[I] = va
    
    uwv[I] = uw 
    vwv[I] = vw
    
    temp[I] = SST

    yil[I+1] = yil[I] + dlat 
    xil[I+1] = xil[I] + dlon / cos((yil[I+1] + yil[I]) / 2*pi / 180) 
    
    
    # Check if out-of-bounds
    ## TODO: find replacements for MATLAB's find() and any() functions in Python
    
    if xil[I+1] > maxLON | xil[I+1] < minLON | yil[I+1] > maxLAT | yil[I+1] < minLAT:
    
        outofbound = 1
        ob = ob + 1
        print('iceberg {} left boundary at timestep {} \n'.format(j, I))

        
    ## now check you didn't send the iceberg on land

    else:  
        
        yi2[1] = find(LAT <= yil[I+1], 1, 'last')
        yi2[2] = find(LAT > yil[I+1], 1, 'first')
        xi2[1] = find(LON <= xil[I+1], 1, 'last')
        xi[2] = find(LON > xil[I+1], 1, 'first')
        
        if any(find(msk(xi2, yi2) == 0)):
            
            yil[I+1] = yil[I]   ## i.e. when I get put within one grid box of land
            xil[I+1] = xil[I]   ## I assume the iceberg don't move, until it doesn't happen anymore
    

In [121]:
LON[seed_X[200]]

array([[ 21.625],
       [ 21.625],
       [ 21.625],
       [ 21.625],
       [ 21.625],
       [ 21.625]], dtype=float32)

In [122]:
def iceberg_shell():
    
    # Loop over individual initial iceberg size classes
    ## (classification from Bigg et al 1997)

    bvec = np.arange(1,10)  # vector of which size classes to compute - has to be [1,10]

    for bb in bvec:
        
        bergsize = bb   # current berg size class
        print('run bergsize B#{} \n'.format(bergsize))
        
        
        # Set output arrays
        
        ## Lat and lon
        XIL = np.empty((trajnum, nt)); XIL[:] = np.NAN; YIL = XIL
        
        ## Vol and dvol
        VOL = XIL; DVOL = VOL   
        
        ## Zonal velocities
        UI = UA = UW = XIL           
        
        ## Merid velocities
        VI = VA = VW = XIL            
        
        ## SST
        TE = XIL                      
        
        ## Melt 
        Memat = Mvmat = Mbmat = XIL  

        
        # Initialize the iceberg
        
        L = bergdims[bergsize,0]
        W = bergdims[bergsize,1]
        H = bergdims[bergsize,2]

        
        # Run drift and melt
        
        tic()
        
        mm = 0 
        ss = 0 
        ob = 0
        
        for j in np.arange(1, trajnum):
            
            if np.mod(j,10) == 0: 
               
                toc() 
                print('{} trajectories computed \n'.format(j))

                      
            # Pick a random trajectory start time (of Input field)
            ## Note: not the best setup for validation runs where you may want specific trajectories to compare!)
            
            ts = np.random.randint(0,round(startrange))
            tts = ts * tres  # trajectory start time (of model)
            lt = nt - tts   # trajectory run length

            
            # Initialize output vectors
            
            xil = np.empty((1, lt)); xil[:] = np.NAN 
            yil = v = dv = xil
            uiv = uav = uwv = viv = vav = vwv = temp = Mev = Mvv = Mbv = v

                      
            # Pick random grid seeding location (same note as above applies)
            
            randoX = np.random.randint(1, seed_X[:,1].size); randoY = np.random.randint(1, seed_Y[:,1].size)
            yig = seed_Y[randoY]; xig = seed_X[randoX]  # 6x1 arrays

            
            # Set initial conditions
            
            ## Lat and lon
            print(xil[0,0])
            xil[0] = LON[xig]; yil[1] = LAT[yig]
            
            ## Berg dims
            l = L * ones[1,lt] 
            w = l * W / L 
            h = l * H / L
            
            ## Berg volume and dvol
            v[1] = L * W * H 
            dv[1] = 0

            ## Count
            i = 0 
            outofbound = 0 
            melted = 0
                    
                
            # Integrate while the iceberg is in the domain and not melted and over the time period specified above
            
            while outofbound == 0 & melted == 0 & i < lt - 1:
                
                i = i + 1
                
                # This is only required if you change params seasonally
                day_yr = ts + i

                drifting()
                
                melting()
                
            
            ind = np.arange(1,i+1)

            
            # Store output into caps variables
            
            ## Trajectories
            XIL[j, ind] = xil[ind]; YIL[j, ind] = yil[ind]
            
            ## Volume
            VOL[j, ind] = v[ind]; DVOL[j, ind] = dv[ind]     

            ## Ice velocities
            UI[j, ind] = uiv[ind]; VI[j, ind] = viv[ind] 
            
            ## Air velocities
            UA[j, ind] = uav[ind]; VA[j, ind] = vav[ind]
            
            ## Water velocities
            UW[j, ind] = uwv[ind]; VW[j, ind] = vwv[ind]
            
            ## SST values
            TE[j, ind] = temp[ind]                     

            ## Melt rates
            Memat[j, ind] = Mev; Mvmat[j, ind] = Mvv; Mbmat[j, ind] = Mbv
            

        # Print message to user
        
        print('{} icebergs died, {} lived, {} left the domain \n'.format(mm,ss,ob))
        
        
        # Save each bergsize trajectories file to output directory

        sio.savemat(outloc + '{}_B{}_full'.format(modelshort, bb),'XIL','YIL','VOL','DVOL','UI','VI','UA','VA','UW','VW','TE','Memat','Mvmat','Mbmat') 


In [123]:
iceberg_shell()

run bergsize B#1 

nan


ValueError: could not broadcast input array from shape (6,1) into shape (231)

In [ ]:
import scipy.io as sio

bergdims = sio.loadmat('/home/evankielley/WagnerModel/Model/bergdims.mat')
bergdims = bergdims['bergdims']
bergdims

In [ ]:
bergdims = sio.loadmat('/home/evankielley/WagnerModel/output/ECCO_20th/E2_B1_full.mat')
bergdims = bergdims['YIL']
bergdims